In [7]:
import torch
Id1 = torch.eye(1).double()
print(Id1)

tensor([[1.]], dtype=torch.float64)


In [5]:
Id1.ndim

2

In [130]:
class MesNet(torch.nn.Module):
        def __init__(self):
            super(MesNet, self).__init__()
            self.beta_measurement = 3*torch.ones(2)
            self.tanh = torch.nn.Tanh()

            self.cov_net = torch.nn.Sequential(torch.nn.Conv1d(1, 5, 1),
                       torch.nn.ReplicationPad1d(4),
                       torch.nn.ReLU(),
                       torch.nn.Dropout(p=0.5),
                       torch.nn.Conv1d(5, 1, 1, dilation=3),
                       torch.nn.ReplicationPad1d(4),
                       torch.nn.ReLU(),
                       torch.nn.Dropout(p=0.5),
                       )
            "CNN for measurement covariance"
            self.cov_lin = torch.nn.Sequential(torch.nn.Linear(19,1),
                                              torch.nn.Tanh(),
                                              )
            self.cov_lin[0].bias.data[:] /= 100
            self.cov_lin[0].weight.data[:] /= 100

        def forward(self, u):
            y_cov = self.cov_net(u)
            z_cov = self.cov_lin(y_cov)
            z_cov_net = self.beta_measurement.squeeze(0)*z_cov
            #measurements_covs = (iekf.cov0_measurement.unsqueeze(0) * (10**z_cov_net))
            return z_cov_net

In [131]:
model = MesNet()

In [132]:
input_data = torch.tensor([[6, 6, 2]], dtype=torch.float32, requires_grad=True)

# Perform a forward pass through the model
output = model(input_data)

In [133]:
output

tensor([[-0.0176, -0.0176]], grad_fn=<MulBackward0>)

In [5]:
import os
import glob
path_data_base= "C:\\Users\\HP\\Desktop\\rawdata1"
date_dirs = os.listdir(path_data_base)
for n_iter, date_dir in enumerate(date_dirs):
    # get access to each sequence
    print('1')
    path1 = os.path.join(path_data_base, date_dir)
    if not os.path.isdir(path1):
        print('1.1')
        continue
    date_dirs2 = os.listdir(path1)
    print('2')

    for date_dir2 in date_dirs2:
        path2 = os.path.join(path1, date_dir2)
        print('2.1')
        if not os.path.isdir(path2):
            print('2.2')
            continue
        # read data
        oxts_files = sorted(glob.glob(os.path.join(path2, 'oxts', 'data', '*.txt')))
        print(oxts_files)

1
2
2.1
['C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000000.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000001.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000002.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000003.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000004.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000005.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000006.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000007.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_26_drive_0001_sync\\oxts\\data\\0000000008.txt', 'C:\\Users\\HP\\Desktop\\rawdata1\\2011_09_26\\2011_09_

In [ ]:
class MesNet(torch.nn.Module):
        def __init__(self):
            super(MesNet, self).__init__()
            self.beta_measurement = 3*torch.ones(2).double()
            self.tanh = torch.nn.Tanh()

            self.cov_net= torch.nn.Sequential(
                    torch.nn.Conv1d(6, 32, 5),
                    torch.nn.ReplicationPad1d(4),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=0.5),
                    torch.nn.Conv1d(32, 32, 5, dilation=3),
                    torch.nn.ReplicationPad1d(4),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=0.5),
                    ).double()
            "CNN for measurement covariance"
            self.cov_lin = torch.nn.Sequential(torch.nn.Linear(32, 2),
                                              torch.nn.Tanh(),
                                              ).double()
            self.cov_lin[0].bias.data[:] /= 100
            self.cov_lin[0].weight.data[:] /= 100

        def forward(self, u, iekf):
            y_cov = self.cov_net(u).transpose(0, 2).squeeze()
            z_cov = self.cov_lin(y_cov)
            z_cov_net = self.beta_measurement.unsqueeze(0)*z_cov
            measurements_covs = (iekf.cov0_measurement.unsqueeze(0) * (10**z_cov_net))
            return measurements_covs

In [1]:
10**2

100

In [ ]:
class MesNet(torch.nn.Module):
        """
        Measurement Network (MesNet) for the Implicit Extended Kalman Filter (IEKF).

        This network takes the input sensor data and predicts the measurement covariance
        for the IEKF. The input data consists of the sensor readings, and the output is
        the predicted measurement covariance.

        Parameters:
            None

        Attributes:
            beta_measurement (torch.Tensor): Scaling factor for the predicted covariance.
            tanh (torch.nn.Tanh): Hyperbolic tangent activation function.

            Convolutional Neural Network (CNN) Layers:
                cov_net (torch.nn.Sequential): CNN layers for processing sensor data.
                cov_lin (torch.nn.Sequential): Linear layer for predicting the covariance.

        Input:
            u (torch.Tensor): Input sensor data tensor with shape (batch_size, num_features=6, sequence_length=6000).
            num_features = 6D pose (acc and gyro in x,y,z direction)
        Output:
            - measurements_covs (torch.Tensor): Predicted measurement covariance tensor with shape (batch_size, 2),
            where 2 represents the two elements of the covariance matrix (variance for gyro and acc).
          """
        def __init__(self):
            super(MesNet, self).__init__()
            self.beta_measurement = 3*torch.ones(2).double()
            self.tanh = torch.nn.Tanh()

            self.cov_net1 = torch.nn.Sequential(
                    torch.nn.Conv1d(6, 32, 5),
                    torch.nn.ReplicationPad1d(4),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=0.5)
            ).double()
            self.cov_net2 = torch.nn.Sequential(
                    torch.nn.Conv1d(32, 32, 5),
                    torch.nn.ReplicationPad1d(4),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=0.5)
            ).double()
            self.cov_net3 = torch.nn.Sequential(
                    torch.nn.Conv1d(32, 32, 5, dilation=3),
                    torch.nn.ReplicationPad1d(4),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=0.5),
                    ).double()
            "CNN for measurement covariance"
            self.cov_lin = torch.nn.Sequential(torch.nn.Linear(32, 2), 
                                              torch.nn.Tanh(),
                                              ).double()
            self.cov_lin[0].bias.data[:] /= 100
            self.cov_lin[0].weight.data[:] /= 100

        def forward(self, u, iekf):
            #print(f"input to network is{u} and shape is {u.shape}")
            y_cov = self.cov_net1(u)
            y_cov = self.cov_net2(y_cov)
            y_cov = self.cov_net3(y_cov).transpose(0, 2).squeeze()
            #print(f"output of conv_net is{y_cov} and shape is {y_cov.shape}")
            z_cov = self.cov_lin(y_cov)
            #print(f"output of conv_lin is{z_cov} and shape is {z_cov.shape}")
            z_cov_net = self.beta_measurement.unsqueeze(0)*z_cov
            #print(f"output of beta mesurments is{z_cov_net} and shape is {z_cov_net.shape}")
            measurements_covs = (iekf.cov0_measurement.unsqueeze(0) * (10**z_cov_net)) 
            """ (10**z_cov_net) apply exponentiation with covariance matrices to ensure positive-definiteness.
                The diagonal elements of a covariance matrix represent the variances of individual random variables.
                Variance is a measure of the spread or dispersion of a random variable. A negative variance would not make sense in this context."""
            #print(f"mesurment cov is{measurements_covs} and shape is {measurements_covs.shape}")
            return measurements_covs
        
def train_filter(args, dataset):

    """
    Sets up and prepares the IEKF filter for training.
    Calls prepare_filter, prepare_loss_data, and save_iekf.
    Initializes optimizer and enters the training loop.
    """
    
    iekf = prepare_filter(args, dataset)
    prepare_loss_data(args, dataset)
    save_iekf(args, iekf)
    optimizer = set_optimizer(iekf)
    start_time = time.time()

    for epoch in range(1, args.epochs + 1):
        train_loop(args, dataset, epoch, iekf, optimizer, args.seq_dim)
        save_iekf(args, iekf)
        print("Amount of time spent for 1 epoch: {}s\n".format(int(time.time() - start_time)))
        start_time = time.time()

def train_loop(args, dataset, epoch, iekf, optimizer, seq_dim):
    """
    Performs training for each dataset in the training set.
    Calls "mini_batch_step" for each dataset to compute loss.
    Updates model parameters using the optimizer.
    """
    loss_train = 0
    optimizer.zero_grad()
    for i, (dataset_name, Ns) in enumerate(dataset.datasets_train_filter.items()):
        t, ang_gt, p_gt, v_gt, u, N0 = prepare_data_filter(dataset, dataset_name, Ns,
                                                                  iekf, seq_dim)

        loss = mini_batch_step(dataset, dataset_name, iekf,
                               dataset.list_rpe[dataset_name], t, ang_gt, p_gt, v_gt, u, N0)

        if (loss == -1 or torch.isnan(loss)):
            cprint("{} loss is invalid".format(i), 'yellow')
            continue
        elif loss > max_loss:
            cprint("{} loss is too high {:.5f}".format(i, loss), 'yellow')
            continue
        else:
            loss_train += loss
            cprint("{} loss: {:.5f}".format(i, loss))

    if loss_train == 0: 
        return 
    loss_train.backward()  # loss_train.cuda().backward()  
    g_norm = torch.nn.utils.clip_grad_norm_(iekf.parameters(), max_grad_norm)
    if np.isnan(g_norm) or g_norm > 3*max_grad_norm:
        cprint("gradient norm: {:.5f}".format(g_norm), 'yellow')
        optimizer.zero_grad()

    else:
        optimizer.step()
        optimizer.zero_grad()
        cprint("gradient norm: {:.5f}".format(g_norm))
    print('Train Epoch: {:2d} \tLoss: {:.5f}'.format(epoch, loss_train))
    return loss_train
